In [1]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

/home/bongo/tensorflow/tfl/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/bongo/tensorflow/tfl/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If y

### Dataset variables

In [2]:
label_map = {1: 'drone'} 
DATASET = 'workspace/images/dataset'
#DATASET = 'workspace/images/dataset2'

train_images_dir = os.path.join(DATASET, 'train', 'images')
train_annotations_dir = os.path.join(DATASET, 'train', 'annotations')
val_images_dir = os.path.join(DATASET, 'validation', 'images')
val_annotations_dir = os.path.join(DATASET, 'validation', 'annotations')
test_images_dir = os.path.join(DATASET, 'test', 'images')
test_annotations_dir = os.path.join(DATASET, 'test', 'annotations')

### DataLoader

In [3]:
train_data = object_detector.DataLoader.from_pascal_voc(train_images_dir, train_annotations_dir, label_map=label_map)
validation_data = object_detector.DataLoader.from_pascal_voc(val_images_dir, val_annotations_dir, label_map=label_map)
test_data = object_detector.DataLoader.from_pascal_voc(test_images_dir, test_annotations_dir, label_map=label_map)

In [4]:
print(f'train count: {len(train_data)}')
print(f'validation count: {len(validation_data)}')
print(f'test count: {len(test_data)}')

train count: 1350
validation count: 58
test count: 58


### Training

In [5]:
spec = model_spec.get('efficientdet_lite0')

2023-10-10 20:48:01.589757: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 20:48:03.038555: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4489 MB memory:  -> device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:03:00.0, compute capability: 6.1


In [5]:
spec = object_detector.EfficientDetLite0Spec(
    hparams={
        'num_classes': 1,
        'alpha': 0.5,
        }  
)

2023-10-11 17:34:48.863950: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-11 17:34:50.304406: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4030 MB memory:  -> device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:03:00.0, compute capability: 6.1


In [5]:
spec = object_detector.EfficientDetLite0Spec()

2023-10-08 16:27:32.209744: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-08 16:27:33.653605: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4711 MB memory:  -> device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:03:00.0, compute capability: 6.1


In [6]:
model = object_detector.create(train_data=train_data, 
                               model_spec=spec, 
                               validation_data=validation_data, 
                               epochs=150, 
                               batch_size=10, 
                               train_whole_model=True)

Epoch 1/150


2023-10-11 17:35:28.303033: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


135/135 [==============================] - 72s 300ms/step - det_loss: 2.0761 - cls_loss: 1.2265 - box_loss: 0.0170 - reg_l2_loss: 0.0643 - loss: 2.1404 - learning_rate: 0.0196 - gradient_norm: 7.1590 - val_det_loss: 7.1207 - val_cls_loss: 5.5896 - val_box_loss: 0.0306 - val_reg_l2_loss: 0.0667 - val_loss: 7.1874
Epoch 2/150
135/135 [==============================] - 38s 284ms/step - det_loss: 1.8361 - cls_loss: 1.1570 - box_loss: 0.0136 - reg_l2_loss: 0.0690 - loss: 1.9051 - learning_rate: 0.0312 - gradient_norm: 4.4680 - val_det_loss: 5.3208 - val_cls_loss: 3.4477 - val_box_loss: 0.0375 - val_reg_l2_loss: 0.0708 - val_loss: 5.3916
Epoch 3/150
135/135 [==============================] - 39s 286ms/step - det_loss: 1.5038 - cls_loss: 0.9337 - box_loss: 0.0114 - reg_l2_loss: 0.0717 - loss: 1.5755 - learning_rate: 0.0312 - gradient_norm: 3.2645 - val_det_loss: 2.0163 - val_cls_loss: 1.3532 - val_box_loss: 0.0133 - val_reg_l2_loss: 0.0726 - val_loss: 2.0889
Epoch 4/150
135/135 [=============

In [7]:
model.evaluate(test_data)

2023-10-11 19:31:14.377488: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.75GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


1/1 [==============================] - 12s 12s/step



{'AP': 0.4642795,
 'AP50': 0.9671321,
 'AP75': 0.33415407,
 'APs': 0.3524129,
 'APm': 0.52810574,
 'APl': 0.8,
 'ARmax1': 0.537931,
 'ARmax10': 0.59827584,
 'ARmax100': 0.6103448,
 'ARs': 0.4952381,
 'ARm': 0.6722222,
 'ARl': 0.8,
 'AP_/drone': 0.4642795}

### Export Model

In [8]:
TFLITE_FILENAME = 'efficientdet-lite-drone-150-alpha05.tflite'
LABELS_FILENAME = 'drone-labels.txt'

In [ ]:
model.export(export_dir='workspace/models', tflite_filename=TFLITE_FILENAME, label_filename=LABELS_FILENAME,
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

2023-10-11 19:31:26.603276: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2023-10-11 19:31:58.966698: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2023-10-11 19:32:05.253792: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-10-11 19:32:05.253851: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-10-11 19:32:05.267591: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp_ft5c4v_
2023-10-11 19:32:05.380449: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-10-11 19:32:05.380487: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if pre

### Test tflite model

In [10]:
import os
model.evaluate_tflite(os.path.join('workspace','models',TFLITE_FILENAME), test_data)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


58/58 [==============================] - 125s 2s/step



{'AP': 0.41383272,
 'AP50': 0.8849029,
 'AP75': 0.31108236,
 'APs': 0.27866486,
 'APm': 0.5016788,
 'APl': 0.8,
 'ARmax1': 0.49655172,
 'ARmax10': 0.51724136,
 'ARmax100': 0.5224138,
 'ARs': 0.40952381,
 'ARm': 0.58055556,
 'ARl': 0.8,
 'AP_/drone': 0.41383272}

In [32]:
! cd workspace/models && edgetpu_compiler $TFLITE_FILENAME -d --num_segments=1

Edge TPU Compiler version 16.0.384591198
Searching for valid delegate with step 1
Try to compile segment with 267 ops
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 3209 ms.

Input model: efficientdet-lite-drone-50-alpha05.tflite
Input size: 4.24MiB
Output model: efficientdet-lite-drone-50-alpha05_edgetpu.tflite
Output size: 5.57MiB
On-chip memory used for caching model parameters: 4.21MiB
On-chip memory remaining for caching model parameters: 3.29MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 267
Operation log: efficientdet-lite-drone-50-alpha05_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations tha